In [1]:
import openmeteo_requests
import requests_cache
import pandas as pd

from retry_requests import retry
from datetime import datetime, timedelta

In [2]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

In [3]:
VAN_LAT = 49.2497
VAN_LONG = -123.1193
START_DATE = "1990-01-01"
END_DATE = (datetime.now() - timedelta(days = 7)).strftime('%Y-%m-%d')
RETRIEVE_COLS = ["weather_code", "temperature_2m_max", "temperature_2m_min", "temperature_2m_mean", "apparent_temperature_max", "apparent_temperature_min", "apparent_temperature_mean", "sunrise", "sunset", "precipitation_sum", "rain_sum", "snowfall_sum", "precipitation_hours", "wind_speed_10m_max", "wind_gusts_10m_max", "wind_direction_10m_dominant", "shortwave_radiation_sum", "et0_fao_evapotranspiration"]

In [4]:
# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": VAN_LAT,
	"longitude": VAN_LONG,
	"start_date": START_DATE,
	"end_date": END_DATE,
	"daily": RETRIEVE_COLS,
    "timezone": "auto"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°E {response.Longitude()}°N")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

Coordinates 49.244285583496094°E -123.13357543945312°N
Elevation 73.0 m asl
Timezone b'America/Vancouver' b'PST'
Timezone difference to GMT+0 -28800 s


In [5]:
# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_weather_code = daily.Variables(0).ValuesAsNumpy()
daily_temperature_2m_max = daily.Variables(1).ValuesAsNumpy()
daily_temperature_2m_min = daily.Variables(2).ValuesAsNumpy()
daily_temperature_2m_mean = daily.Variables(3).ValuesAsNumpy()
daily_apparent_temperature_max = daily.Variables(4).ValuesAsNumpy()
daily_apparent_temperature_min = daily.Variables(5).ValuesAsNumpy()
daily_apparent_temperature_mean = daily.Variables(6).ValuesAsNumpy()
daily_sunrise = daily.Variables(7).ValuesAsNumpy()
daily_sunset = daily.Variables(8).ValuesAsNumpy()
daily_precipitation_sum = daily.Variables(9).ValuesAsNumpy()
daily_rain_sum = daily.Variables(10).ValuesAsNumpy()
daily_snowfall_sum = daily.Variables(11).ValuesAsNumpy()
daily_precipitation_hours = daily.Variables(12).ValuesAsNumpy()
daily_wind_speed_10m_max = daily.Variables(13).ValuesAsNumpy()
daily_wind_gusts_10m_max = daily.Variables(14).ValuesAsNumpy()
daily_wind_direction_10m_dominant = daily.Variables(15).ValuesAsNumpy()
daily_shortwave_radiation_sum = daily.Variables(16).ValuesAsNumpy()
daily_et0_fao_evapotranspiration = daily.Variables(17).ValuesAsNumpy()

In [6]:
daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s").strftime('%Y-%m-%d'),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s").strftime('%Y-%m-%d'),
	freq = pd.Timedelta(days = 1),
	inclusive = "left"
)}
daily_data["weather_code"] = daily_weather_code
daily_data["temperature_2m_max"] = daily_temperature_2m_max
daily_data["temperature_2m_min"] = daily_temperature_2m_min
daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
daily_data["apparent_temperature_max"] = daily_apparent_temperature_max
daily_data["apparent_temperature_min"] = daily_apparent_temperature_min
daily_data["apparent_temperature_mean"] = daily_apparent_temperature_mean
daily_data["sunrise"] = daily_sunrise
daily_data["sunset"] = daily_sunset
daily_data["precipitation_sum"] = daily_precipitation_sum
daily_data["rain_sum"] = daily_rain_sum
daily_data["snowfall_sum"] = daily_snowfall_sum
daily_data["precipitation_hours"] = daily_precipitation_hours
daily_data["wind_speed_10m_max"] = daily_wind_speed_10m_max
daily_data["wind_gusts_10m_max"] = daily_wind_gusts_10m_max
daily_data["wind_direction_10m_dominant"] = daily_wind_direction_10m_dominant
daily_data["shortwave_radiation_sum"] = daily_shortwave_radiation_sum
daily_data["et0_fao_evapotranspiration"] = daily_et0_fao_evapotranspiration

In [8]:
df_van_weather = pd.DataFrame(data = daily_data)
df_van_weather = df_van_weather.set_index('date')
display(df_van_weather)

,weather_code,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,sunrise,sunset,precipitation_sum,rain_sum,snowfall_sum,precipitation_hours,wind_speed_10m_max,wind_gusts_10m_max,wind_direction_10m_dominant,shortwave_radiation_sum,et0_fao_evapotranspiration
date,,,,,,,,,,,,,,,,,,
1990-01-01,51.0,5.7095,2.0095,3.903250,1.759364,-2.227180,0.155060,0,0,1.600000,1.600000,0.00,7.0,18.250259,35.279999,222.906708,4.09,0.504341
1990-01-02,71.0,3.0595,0.0095,1.959500,-0.347375,-2.789511,-1.547469,0,0,1.000000,0.700000,0.21,5.0,12.261158,28.799999,171.749664,3.29,0.467342
1990-01-03,73.0,4.4095,1.9595,2.934500,1.055596,-1.572106,-0.585330,0,0,11.400002,10.600001,0.56,24.0,17.555307,34.560001,141.703812,1.89,0.201307
1990-01-04,61.0,7.3595,3.2095,4.917833,3.778481,0.573077,1.792292,0,0,12.599999,12.599999,0.00,14.0,18.806337,38.160000,162.864502,2.43,0.277709
1990-01-05,63.0,8.3595,3.6595,5.892833,4.432674,0.000019,2.062605,0,0,17.699999,17.699999,0.00,17.0,32.919827,64.439995,159.804855,0.64,0.168201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-02,63.0,13.0305,7.6305,9.984666,11.770529,5.137917,7.995744,0,0,53.900005,53.900005,0.00,18.0,21.243050,34.919998,86.653229,3.12,0.478838
2023-11-03,61.0,11.7305,7.3305,9.782582,9.703583,5.214563,7.581160,0,0,4.700000,4.700000,0.00,7.0,20.469410,33.839996,80.872467,3.09,0.572418
2023-11-04,65.0,14.1805,9.2805,11.647167,13.198673,7.234596,9.908348,0,0,32.999996,32.999996,0.00,15.0,21.437386,38.880001,106.656235,3.54,0.508575


In [10]:
df_van_weather.to_csv(f'van_weather_{START_DATE}_{END_DATE}.csv')